In [43]:
from itertools import product
from random import shuffle
from tqdm import tqdm
from language_models.character_frequency_kld_language_model import CharacterFrequencyKLDLanguageModel

In [44]:
# Imports de Enigma
from enigma.enigma import Enigma
from enigma.plugboard import Plugboard
from enigma.reflectors.reflector import Reflector
from enigma.reflectors.reflector_b import ReflectorB
from enigma.reflectors.reflector_b_thin import ReflectorBThin
from enigma.reflectors.reflector_c import ReflectorC
from enigma.rotors.rotor import Rotor
from enigma.rotors.rotor_I import RotorI
from enigma.rotors.rotor_II import RotorII
from enigma.rotors.rotor_III import RotorIII
from enigma.rotors.rotor_IV import RotorIV

In [45]:
ENGLISH_ALPHABET_CHARACTERS = "abcdefghijklmnopqrstuvwxyz"
ENGLISH_ALPHABET_LEN = len(ENGLISH_ALPHABET_CHARACTERS)
# TODO: Is this a parameter?
HARDCODED_PERIOD = 26

with open('books/Frankenstein.txt', 'r') as book:
    frankestein_book = text = [c for c in "".join(book.readlines()).lower() if c in ENGLISH_ALPHABET_CHARACTERS]

PLUGBOARD_TUPLES = [('A', 'F'), ('G', 'H'), ('Y', 'S'), ('M', 'T')]

In [58]:
# Me defino una enigma
enigma = Enigma(reflector=ReflectorB(),
                        plugboard=Plugboard(PLUGBOARD_TUPLES),
                        rotors=[RotorIII(offset=2),
                                RotorII(offset=3),
                                RotorI(offset=5)])

In [59]:

# Modelo de lenguaje basado en la frecuencia de caracteres en un libro en ingles
with open('books/Alices_Adventures_in_Wonderland.txt', 'r') as book:
    freq_kld_model = CharacterFrequencyKLDLanguageModel(book.read())   

with open('books/Frankenstein.txt', 'r') as book:
    all_book = "".join(book.readlines()).lower()
    frankestein_book = "".join([c for c in all_book if c in ENGLISH_ALPHABET_CHARACTERS])

In [64]:
# Me genero todas las posibles combinaciones de los offsets de los rotores
def crack(language_model, encrypted_message):
    offset_rotor_combinations = list(product(range(ENGLISH_ALPHABET_LEN), repeat = 3))
    kb_divergences = {}
    i=0
    for offsets in tqdm(range(26**3), total=26**3):
        off1 = offsets % 26
        off2 = (offsets//26) % 26
        off3 = (offsets//(26**2)) % 26
        enigma_with_given_offset = Enigma(ReflectorB(),
                                          plugboard=Plugboard(PLUGBOARD_TUPLES),
                                          rotors=[
                                              RotorIII(offset=off1),
                                              RotorII(offset=off2),
                                              RotorI(offset=off3)])
        
        decrypted_message = enigma_with_given_offset.decrypt(encrypted_message)
        kb_divergences[(off1, off2, off3)] = freq_kld_model.fitness(decrypted_message)
        
        i+=1
    sorted_divergences = sorted(kb_divergences.items(), key=lambda x: x[1], reverse = True)
    return sorted_divergences[:50]

In [65]:
%%time
type(frankestein_book)
crack(freq_kld_model, enigma.encrypt(frankestein_book[100:200]))

100%|██████████| 17576/17576 [00:14<00:00, 1247.71it/s]

CPU times: user 13.9 s, sys: 115 ms, total: 14 s
Wall time: 14.1 s


[((2, 3, 5), -0.08709331423756198),
 ((18, 15, 10), -0.1865177273234864),
 ((6, 4, 14), -0.20347674459395415),
 ((21, 23, 11), -0.20656038456261658),
 ((14, 9, 9), -0.22710153117373297),
 ((18, 2, 4), -0.2278287699201741),
 ((3, 12, 19), -0.22857927346877632),
 ((14, 24, 9), -0.23090324535932247),
 ((1, 0, 8), -0.2309191184878025),
 ((9, 11, 12), -0.230929604284494),
 ((3, 7, 10), -0.23255072543842153),
 ((1, 12, 14), -0.23268682641615207),
 ((10, 20, 12), -0.23521032991996804),
 ((14, 20, 20), -0.23704545509246253),
 ((25, 20, 5), -0.23851493493261194),
 ((4, 9, 2), -0.23950688390570812),
 ((22, 15, 20), -0.24159852719862077),
 ((21, 12, 10), -0.24366586491296374),
 ((9, 21, 13), -0.24384920963832635),
 ((8, 6, 16), -0.2440916047857562),
 ((18, 23, 9), -0.24414652750032081),
 ((9, 24, 1), -0.24505864487776777),
 ((13, 10, 22), -0.24549225098024052),
 ((15, 11, 10), -0.24664809398654888),
 ((3, 17, 21), -0.24675917925335739),
 ((18, 10, 11), -0.24854835795276325),
 ((2, 2, 21), -0.2496